In [1]:
from datetime import datetime
h=datetime.today().month      # 현재 월 가져오기
m=datetime.today().day        # 현재 일 가져오기

hm='0'+str(h)+str(m)


In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
import os
import matplotlib.pyplot as plt

In [3]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, Concatenate
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


Using TensorFlow backend.


# 데이터 불러오기

In [4]:
# 데이터 불러오기
train = pd.read_csv('/home/lab21/data/train.csv', index_col=0)
train_2 = pd.read_csv('/home/lab21/data/train_x_0.2_99.8.csv', index_col=0)

test = pd.read_csv('/home/lab21/data/test.csv', index_col=0)
sample_submission = pd.read_csv('/home/lab21/data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
#x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = train_2.drop(columns=['fiberID'], axis=1)

test_x = test.drop(columns=['fiberID'],axis=1)

In [5]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [6]:
x=x.astype('float32')
test_x=test_x.astype('float32')

# Scaler

In [7]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x)
x_train=scaler2.transform(x)
test_x =scaler2.transform(test_x)

# Learning Scheduler

In [8]:
import math
from keras.callbacks import Callback
class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

# GELU function

In [9]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf

class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
        
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

get_custom_objects().update({'gelu': Gelu(gelu)})

# Variables

In [10]:
lr = 0.0008
lr_d = 0
patience =  300
drop = 0.2
nepoch = 800

In [11]:
skfold = 7
skf = StratifiedKFold(n_splits=skfold, shuffle=True, random_state=42)

In [12]:
import datetime
start=datetime.datetime.now()
print(start)

2020-02-20 10:24:15.443186


# Deep Learning Model
- EarlyStop, CosineScheduler, CheckPoint

In [ ]:
for enum, (train_index,valid_index) in enumerate(skf.split(x,y)):
    
    file_path = f"/home/lab21/20200220/model/best_ensembl2_{enum}.hdf5"
    check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1,save_best_only=True, mode="min")
    early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
    cosine_scheduler = CosineAnnealingScheduler(T_max=50, eta_max=6e-4, eta_min=3e-5)
    swa = SWA(start_epoch=50, lr_schedule='manual', swa_lr=3e-4,  verbose=1,batch_size=256)

      
    skf_x_tr = x[train_index]
    skf_y_tr = y[train_index]
    
    skf_x_val = x[valid_index]
    skf_y_val = y[valid_index]

    inps= Input(shape = (20,))
    la = Dense(128)(inps)
    la = BatchNormalization()(la)
    la = Activation(gelu)(la)
    x = Dropout(drop)(la)

    for i in range(3):
        xp = Dense(128)(x)

        x = Dense(256)(x)
        x = BatchNormalization()(x)
        x = Activation(gelu)(x)
        x= Dropout(0.25)(x)

        xq= Dense(128)(x)

        x= Concatenate()([xp,xq])
        x = BatchNormalization()(x)
        x = Activation(gelu)(x)

    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation(gelu)(x)
    x = Dropout(drop)(x)

    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation(gelu)(x)
    x = Dropout(drop)(x)
    
    outs=Dense(19,activation='softmax',name='output')(x)

    models = Model(inputs=inps, outputs=outs)
    
    
    models.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=lr,decay=lr_d),metrics=['accuracy'])
    models.fit(skf_x_tr, skf_y_tr, batch_size=256, epochs=nepoch, validation_data = [skf_x_val, skf_y_val],
              verbose=2, callbacks=[early_stop,check_point,cosine_scheduler,swa])
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 171413 samples, validate on 28578 samples
Model uses batch normalization. SWA will require last epoch to be a forward pass and will run with no learning rate
Epoch 1/800
 - 41s - loss: 0.6990 - acc: 0.7800 - val_loss: 1.5716 - val_acc: 0.6123

Epoch 00001: val_loss improved from inf to 1.57157, saving model to /home/lab21/20200220/model/best_ensembl2_0.hdf5
Epoch 2/800
 - 37s - loss: 0.5305 - acc: 0.8203 - val_loss: 0.9898 - val_acc: 0.6948

Epoch 00002: val_loss improved from 1.57157 to 0.98981, saving model to /home/lab21/20200220/model/best_ensembl2_0.hdf5
Epoch 3/800
 - 38s - loss: 0.5024 - acc: 0.8276 - val_loss: 0.8684 - val_acc: 0.6840

Epoch 00003: val_loss improved from 0.98981 to 0.86841, saving model to /home/lab21/20200220/model/best_

In [ ]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)

# Model Predict

In [ ]:
pred_test = np.zeros((len(test_x),19))

for i in range(skfold):
    print(i)    
    model_load = load_model("/home/lab21/20200220/model/best_ensemble2_{}.hdf5".format(i))
    pred = np.array(model_load.predict(test_x))
    pred_test += pred
    
pred_test = pred_test /skfold

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y, train_size=0.8, \
                                                    shuffle=True, random_state=42)

In [ ]:
loss, acc= models.evaluate(x_test,y_test)
print("LOSS: ",loss, "ACC:", acc)

In [ ]:
# 제출 파일 생성
submission = pd.DataFrame(data=pred_test, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('/home/lab21/20200220/csv/submission_ensemble2.csv', index=True)